In [43]:
import pandas as pd
from collections import defaultdict
from collections import Counter
import re
import math
import json

In [44]:
# Cargar el archivo de entrenamiento
with open("chat.txt", "r") as file:
    chat = file.read()

In [46]:
# Procesamiento de del chat para obtener una lista de palabras
lines = chat.split('\n')
words = []

frases_eliminar = ["Creó el grupo", "Se eliminó este mensaje", "Multimedia omitido", "Se editó este mensaje", "Se te añadió al grupo", "Se unió usando el enlace de invitación de este grupo", "Salió del grupo", "Añadió a"]

eliminar = [frase.split() for frase in frases_eliminar]

for line in lines:
    
    if ":" in line:
        message = line.rsplit(":", 1)[-1].strip()
        message_clean = re.findall(r'[\wáéíóúÁÉÍÓÚñÑ]+', message) 
        
        if not message_clean in eliminar: # sacamos los textos que no son mensajes
            words.extend(word.lower() for word in message_clean) # conservamos las palabras pero en minusculas
        

In [47]:
# Cargar el diccionario en español
with open("index.json", "r", encoding="utf-8") as json_file:
    diccionario_esp = json.load(json_file)


In [94]:
def eliminar_tildes(palabra):
    tildes = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u'}
    return ''.join(tildes.get(char, char) for char in palabra)

# Eliminar tildes de las palabras
words_sin_tildes = [eliminar_tildes(word) for word in words]

# Aplicando criterio de todo o nada

# Lista para almacenar las palabras corregidas
words_corr_st = []

# Corregir errores ortográficos
for word in words_sin_tildes:
    # Si la palabra está en el diccionario personalizado se guarda
    if word in diccionario_esp:
        words_corr_st.append(word)

In [49]:
def frases_de_horizonte(horizonte, words):
    '''
    Devuelve un diccionario con un key por palabra del vocabulario y valores las frases de largo horizonte que terminan en la palabra
    '''
    iter = list(set(words))
    frases= {}
    for target in iter: #Tomo las palabras para las que quiero las probab condicionales
        # Length of sublists you want to find
        n = horizonte
        # listas de palabras con la palabra target
        frases[target] = [words[max(i-n,0):i] for i in range(len(words)) if target == words[i]]
    return frases



In [50]:
def cuantos(b, a, frases):
    '''
    Cuenta la cantidad de frases que terminan en a tienen a b
    '''
    frases_con_b = 0
    if a in frases:
        for words in frases[a]:
            if b in words:
                frases_con_b += 1
    return frases_con_b

In [51]:
#Diccionario con las probabilidades a priori de las palabras
def Priori(words):
    '''
    Calcula un diccionario con las probabilidades a priori de cada palabra
    '''
    P = {}
    for palabra in list(set(words)):
        P[palabra] = words.count(palabra)/len(words) #cantidad de ocurrencias de la palabra en todas las instancias
    return P

In [52]:
def Probabilidades_condicionales(words, horizonte, m):
    '''
    Calcula un diccionario con las probabilidades condicionales de cada palabra de estar en una frase de largo horizonte con cada otra palabra
    '''
    PD = {} #Diccionario con las probabilidades condicoinales m-estimador

    frases = frases_de_horizonte(horizonte, words)

    pal_unicas = list(set(words))

    for a in pal_unicas: # P( _ | a)
        PD[a]={}
        itera = pal_unicas.copy()
        itera.remove(a)
        for b in itera: # P( b | a )
            e = cuantos(b,a,frases) #frases con la palabra b que terminan en a
            p = 1/len(pal_unicas) #probabilidad de una palabra
            n = len(frases[a]) #frases que terminan con a
            PD[a][b]= (e + m*p)/(n+m)
    return PD

In [91]:
horizonte = 4
m = 4
PD = Probabilidades_condicionales(words, horizonte, m)
P = Priori(words)

In [92]:
def recomendacion_bayesiana(frase):
    D = frase   # Frase que será ingresada
    
    P_nada = 0.000000001  # valor pequeño por si la palabra no aparece en el diccionario
    h_MAP = "" 
    p_MAP = float('-inf') ##variable que se utilizará para almacenar la probabilidad más alta encontrada hasta ahora.

    for h in P: # me fijo en cada palabra posible
        prome = P[h]
        prob = math.log(P[h])
        for d in D[-horizonte:]:
            prome = prome + PD[h].get(d, P_nada)
            prob = prob + math.log(PD[h].get(d, P_nada)) #log(P(sugerencia))+log(P(palabra1|sugerencia))+...+log(P(palabran|sugerencia))
        if prob > p_MAP:
            promedio = prome # Guardo el promedio de los factores de h_MAP
            h_MAP, p_MAP = h, prob
    promedio = promedio/len(D[-horizonte:])
    print("Promedio de los factores de la sugerencia ",h_MAP," =", promedio)
    return(h_MAP)


In [114]:
horizonte_d = 2
m = 2
PD_dict = Probabilidades_condicionales(words_corr_st, horizonte_d, m)
P_dict = Priori(words_corr_st)

In [115]:
def recomendacion_bayesiana_dict(frase):
    D = frase   # Frase que será ingresada
    
    P_nada = 0.000000001  # valor pequeño por si la palabra no aparece en el diccionario
    h_MAP = "" 
    p_MAP = float('-inf') ##variable que se utilizará para almacenar la probabilidad más alta encontrada hasta ahora.

    for h in P_dict: # me fijo en cada palabra posible
        prome = P_dict[h]
        prob = math.log(P_dict[h])
        for d in D[-horizonte:]:
            prome = prome + PD_dict[h].get(d, P_nada)
            prob = prob + math.log(PD_dict[h].get(d, P_nada)) #log(P(sugerencia))+log(P(palabra1|sugerencia))+...+log(P(palabran|sugerencia))
        if prob > p_MAP:
            promedio = prome # Guardo el promedio de los factores de h_MAP
            h_MAP, p_MAP = h, prob
    promedio = promedio/len(D[-horizonte:])
    print("Promedio de los factores de la sugerencia ",h_MAP," =", promedio)
    return(h_MAP)


In [120]:
##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
diccionario = True #Parametro para sugerir con o sin diccionario
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
      break

    elif palabra == ".":
      print("----- Comenzando frase nueva -----")
      frase = []

    elif palabra == "": # acepta última palabra sugerida
      frase.append(palabra_sugerida)

    else: # escribió una palabra
      frase.append(palabra.lower())

    if frase:
      if diccionario:
        palabra_sugerida = recomendacion_bayesiana_dict(frase)
      else:
        palabra_sugerida = recomendacion_bayesiana(frase)

      frase_propuesta = frase.copy()
      frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

      print(" ".join(frase_propuesta))

Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.
Promedio de los factores de la sugerencia  no  = 0.033930189087056674
saben no
Promedio de los factores de la sugerencia  te  = 0.04524838485796101
saben si te
Promedio de los factores de la sugerencia  cero  = 0.03804583848327669
saben si te cero
Promedio de los factores de la sugerencia  cosas  = 0.011661685814601232
saben si te cero cosas
Promedio de los factores de la sugerencia  quiero  = 0.08444406718682036
saben si te cero cosas quiero
Promedio de los factores de la sugerencia  que  = 0.02482877297598434
saben si te cero cosas quiero que
Promedio de los factores de la sugerencia  yo  = 0.04761500916950444
saben si te cero cosas quiero que yo
Promedio de los factores de la sugerencia  saber  = 0.058775652784000355
saben si te cero cosas quiero q

In [63]:
words2 = words.copy()

In [65]:
##### LOOP CON REENTRENAMIENTO #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
      break

    elif palabra == ".":
      words2.extend(frase) # Agregamos las palabras de la nueva frase a la lista de mensajes
      PD = Probabilidades_condicionales(words2, horizonte, m)
      P = Priori(words2)
      print("----- Comenzando frase nueva -----")
      frase = []

    elif palabra == "": # acepta última palabra sugerida
      frase.append(palabra_sugerida)

    else: # escribió una palabra
      frase.append(palabra.lower())

    if frase:
      palabra_sugerida = recomendacion_bayesiana(frase)

      frase_propuesta = frase.copy()
      frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

      print(" ".join(frase_propuesta))





Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.


Promedio de los factores de la sugerencia  que  = 0.043846352857678114
australopitecus que
Promedio de los factores de la sugerencia  que  = 0.021923176928839057
australopitecus aguaraguazú que
Promedio de los factores de la sugerencia  que  = 0.021923176928839057
australopitecus aguaraguazú intrepidamente que
Promedio de los factores de la sugerencia  que  = 0.021923176928839057
australopitecus aguaraguazú intrepidamente cabilan que
----- Comenzando frase nueva -----
Promedio de los factores de la sugerencia  intrepidamente  = 0.3334147834347437
australopitecus intrepidamente
Promedio de los factores de la sugerencia  intrepidamente  = 0.16670739221737185
australopitecus aguaraguazu intrepidamente
Promedio de los factores de la sugerencia  cabilan  = 0.16670739221737185
australopitecus aguaraguazu intrepidamente cabilan
Promedio de los factores de la sugerencia  pov  = 6.347095721760371e-05
australopitecus aguaraguazu intrepidamente cabilan pov
